In [81]:
#Trying to get number of crags on an individual state page.

import pandas as pd
import requests
from bs4 import BeautifulSoup

url = 'https://www.mountainproject.com/area/105905173/alabama'
page = requests.get(url)


# Getting the webpage, creating a Response object.
response = requests.get(url)
 
# Extracting the source code of the page.
data = response.text
 
# Passing the source code to Beautiful Soup to create a BeautifulSoup object for it.
soup = BeautifulSoup(data, 'lxml')
 
# Extracting all the <a> tags into a list. No ID or class or anything next to hyperlink in HTML
titles = soup.findAll('a')
 
# Extracting text from the the <a> tags, i.e. class titles and put into list. 
final = []
for title in titles:
    final.append(title.text)

df = pd.DataFrame(final)
df.head(5)

,0
0,Your login works with all Adventure Projects s...
1,Connect with Facebook
2,Password help
3,Cancel
4,\r\n REI Accessibility Statement\r\n ...


# So, we've gotten text and titles into a dataframe. Need to remove non-crags.

# Assuming each state page is laid out the same, we can remove 0:46 as the strings in the heading and such.  And then we only need to remove everything after the entry '\n\Previous\n'.  This will give us a dataframe of just the climbing spots on the state page.  The function below finds the location of this entry.  

In [82]:
def getIndexes(dfObj, value):
    ''' Get index positions of value in dataframe i.e. dfObj.'''
 
    listOfPos = list()
    # Get bool dataframe with True at positions where the given value exists
    result = dfObj.isin([value])
    # Get list of columns that contains the value
    seriesObj = result.any()
    columnNames = list(seriesObj[seriesObj == True].index)
    # Iterate over list of columns and fetch the rows indexes where value exists
    for col in columnNames:
        rows = list(result[col][result[col] == True].index)
        for row in rows:
            listOfPos.append((row, col))
    # Return a list of tuples indicating the positions of value in the dataframe
    return listOfPos

In [83]:
#Find location of desired string, remove everything after and including it
cutOff = getIndexes(crags,'\nPrevious\n')
df.drop(df.index[cutOff[0][0]:(len(df))],inplace = True)

In [84]:
#Remove text displayed before climbing spots.
df.drop(df.index[0:46], inplace = True)

In [85]:
#Rename column to 'crags'
df.columns = ['Crags']

In [86]:
df.head()

,Crags
46,BankHead Forest/ Thompson Creek Trail
47,Bankhead Forest/Ship Rock
48,bankhead forest/sipsey picnic area
49,Cherokee Bluffs
50,Chewacla State Park
